In [1]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

Using TensorFlow backend.


In [2]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng23train.csv', encoding="latin-1")
#balancing training data by redundancy
d1=df_train[df_train.task_3==0]
print(len(d1))
d2=df_train[df_train.task_3==1]
print(len(d2))
d1=pd.concat([d1,d1,d1,d1,d1,d1],ignore_index=True)
print(len(d1))
df_train=pd.concat([d1,d2],ignore_index=True)
df_train.head()

220
2041
1320


,text_id,text,task_1,task_2,task_3
0,hasoc_en_135,fck the us tho and fucktrump httpstcowyc6rqw##x,0,0,0
1,hasoc_en_158,@adityarajkaul dukarr ð·ð½ðµð° mullahs...,0,0,0
2,hasoc_en_250,the great thing about all the nazis right wing...,0,0,0
3,hasoc_en_659,what a fucking idiotic thing to do threaten pr...,0,0,0
4,hasoc_en_2685,ð£fuck america and this health care shit the...,0,0,0


In [3]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_3'],test_size=0.15,random_state=140) 

In [4]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [5]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/english_dataset/eng23test.csv', encoding="latin-1") 
test_y = encoder.fit_transform(df_test['task_3'])#encoding the label of testing data

In [6]:
# create a tfidf vectorizer object 
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=25999)
tfidf_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_tfidf =  tfidf_vect.transform(train_x)  #get tfidf vector features for training data
xvalid_tfidf =  tfidf_vect.transform(valid_x)  #get tfidf vector features for validation data
x_tfidf =  tfidf_vect.transform(df_test['text'])  #get tfidf vector features for testing data

In [7]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [8]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc(),x_tfidf.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.81      0.87      0.84       179
           1       0.92      0.89      0.91       326

    accuracy                           0.88       505
   macro avg       0.87      0.88      0.87       505
weighted avg       0.88      0.88      0.88       505

classification report for testing
              precision    recall  f1-score   support

           0       0.26      0.19      0.22        58
           1       0.81      0.86      0.83       230

    accuracy                           0.73       288
   macro avg       0.53      0.53      0.53       288
weighted avg       0.70      0.73      0.71       288

